In [1]:
import numpy as np
from MFEA_lib.tasks.function import Sphere, Ackley, Rosenbrock
from MFEA_lib.model.mfea import MFEA1
from MFEA_lib.operators.CrossOver import SBX_CrossOver
from MFEA_lib.operators.Selection import ElitismSelection
from MFEA_lib.operators.Mutation import Polynomial_Mutation


In [2]:
tasks = [Sphere(30, bound= [-100, 100]), 
        Ackley(50, bound= [-100, 100]), 
        Rosenbrock(20, bound= [-100, 100]), 
        Rosenbrock(30, bound= [-100, 100])
]


In [3]:
model = MFEA1()
model.compile(
    cross_over= SBX_CrossOver(nc = 15),
    mutation= Polynomial_Mutation(nm = 15),
    selection= ElitismSelection()
)
sol, his = model.fit(tasks= tasks, num_generations= 2000, num_inds_each_task= 100, rmp =0.3)

Epoch [100/2000], [=>                  ]   5% ,func_val: [10910.110115675876, 20.86435185877311, 38062907.723847, 703553195.5280896]

Epoch [200/2000], [==>                 ]  10% ,func_val: [858.6181406132811, 20.3067809712663, 217559.8008747807, 5502073.273667617]

Epoch [300/2000], [===>                ]  15% ,func_val: [44.119257876461944, 20.12874487624195, 3459.5322422939707, 18248.60660189169]

Epoch [400/2000], [====>               ]  20% ,func_val: [12.213243913411642, 20.07050204094464, 399.22804392898337, 1580.6524520796643]

Epoch [500/2000], [=====>              ]  25% ,func_val: [4.05646992244508, 20.04037234329093, 149.4963441504369, 345.07342418213534]

Epoch [600/2000], [======>             ]  30% ,func_val: [1.475195288355797, 20.025212451821645, 73.34660625007052, 155.02386542624623]

Epoch [700/2000], [=======>            ]  35% ,func_val: [0.5890103196717931, 20.015357977802378, 41.04928883699965, 75.66055743314351]

Epoch [800/2000], [========>           ]  40% ,f